In [ ]:
import math as m
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import seaborn as sns

# Đọc file chứa dữ liệu và in ra tiêu đề các cột

In [ ]:
data = pd.read_csv('../data/electrical+grid+stability+simulated+data/Data_for_UCI_named.csv')
print(list(data.columns))
print(data.shape)

In [ ]:
data.head()

# Vẽ đồ thị phân bổ dữ liệu stabf và số lượng mẫu

In [ ]:
sns.countplot(x='stabf', data=data, palette='hls')
plt.show()

# Chuyển text cho cột đầu ra thành số 0, 1

In [ ]:
cat_features = ['stabf']
for feature in cat_features:
    print(feature)
    job_list = pd.get_dummies(data[feature], prefix=feature)
    new_data = data.join(job_list)
    data = new_data
    
data.head()

# Xóa bỏ cột chứa đầu vào không dùng

In [ ]:
data_features = data.columns.values.tolist()
print(data_features)
remove_features = ['stabf', 'stabf_unstable']
to_keep_features = [feature for feature in data_features if feature not in remove_features]
data_final = data[to_keep_features]
data_final.head()

# Chuyển dữ liệu thành mảng

In [ ]:
X = np.array(data_final.loc[:, data_final.columns != 'stabf_stable'])
print(X)

In [ ]:
y = np.array(data_final.loc[:, data_final.columns == 'stabf_stable'])
print(y)

# Phân chia tập dữ liệu ra thành 2 tập: tập huấn luyện và tập kiểm tra:

In [ ]:
train_features, test_features, train_targets, test_targets = train_test_split(X, y, test_size=0.3, random_state=0)
print('##### Training and test datasets #####')
print('Training size:', len(train_targets))
print('Test size    :', len(test_targets))

# Biến đổi kích thước ma trận dữ liệu huấn luyện kiểm tra

In [ ]:
train_targets = train_targets.reshape(train_targets.shape[0])
test_targets = test_targets.reshape(test_targets.shape[0])
print(train_targets)

# Vẽ đồ thị

In [ ]:
pd.crosstab(train_targets, train_targets).plot(kind='bar')
plt.title('Histogram of electrical grid stability')
plt.xlabel('Quality')
plt.ylabel('Number of samples')
plt.rc("font", size=14)

In [ ]:
pd.crosstab(test_targets, test_targets).plot(kind='bar')
plt.title('Histogram of electrical grid stability')
plt.xlabel('Quality')
plt.ylabel('Number of samples')
plt.rc("font", size=14)

# Tạo mô hình sử dụng thuật toán SVM, huấn luyện mô hình:

In [ ]:
svmClassifier = SVC(kernel='linear', C = 1e5)
svmClassifier.fit(train_features, train_targets)
print(svmClassifier.coef_)
print(svmClassifier.intercept_)

# Hiển thị độ chính xác của mô hình đối với tập huấn luyện

In [ ]:
train_predictions = svmClassifier.predict(train_features)
print("#### Training - Prediction results of SVM ####")
print("Target labels: ", train_targets)
print("Predictions  : ", train_predictions)

# Hiển thị kết quả đích và kết quả dự đoán của mô hình đối với tập huấn luyện

In [ ]:
accuracy = accuracy_score(train_targets, train_predictions) * 100
print("##### Training - Prediction accuracy of SVM #####")
print("Accuracy of Logistic Regression: ", accuracy)
print(classification_report(train_targets, train_predictions))

# Hiển thị độ chính xác của mô hình đối với tập kiểm tra

In [ ]:
test_predictions = svmClassifier.predict(test_features)
print("#### Test - Prediction results of SVM ####")
print("Target labels: ", test_targets)
print("Predictions  : ", test_predictions)

In [ ]:
accuracy = accuracy_score(test_targets, test_predictions) * 100
print("##### Test - Prediction accuracy of SVM #####")
print("Accuracy of Logistic Regression: ", accuracy)
print(classification_report(test_targets, test_predictions))